In [18]:
import html
import io
import os
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

import requests
import yfinance as yf
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

C:\Users\astalati\AppData\Roaming\Python\Python311\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


#### Load the environment variables

In [19]:
load_dotenv("./.env")
api_endpoint = os.getenv("OpenAiAssistantEp")
api_key = os.getenv("OpenAiAssistantKey")
api_version = os.getenv("OpenAiAssistantVersion")
api_deployment_name = os.getenv("OpenAiGpt4Turbo")

#### Create an AzureOpenAI client

In [7]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

#### Prepare the tools for function calling

In [8]:
tools_list = [
    {"type": "code_interpreter"},
]

In [9]:
dataFolder = "./Data/CSV/FailedBanks"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(dataFolder)
assistant_files = []
for file in arr:
    filePath = dataFolder + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

#### Create an Assistant and a Thread

In [10]:
assistant = client.beta.assistants.create(
    name="Failed Banks Assistant",
    instructions="You are an assistant that can help find information about failed banks. "
    + "Use the provided file only.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

#### Format and display the Assistant Messages for text and images

In [11]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read() 

def print_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

#### Process the user messages

In [12]:
def process_prompt(prompt: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            # Handle function calling and continue processing
            pass
        else:
            time.sleep(5)

#### Process user requests

In [13]:
process_prompt("Create a chart of failed banks by state.")

processing ...
user:
Create a chart of failed banks by state.

assistant:
To create a chart of failed banks by state, I would need relevant data, which may be contained in one of the files you've uploaded. Let's begin by examining the content of the uploaded files to identify which one contains the information we need to generate the chart. I will go through the files one by one and provide a summary of their contents. After identifying the correct file, I will extract the data and create the chart for you.

I will start by examining the first file provided. Please give me a moment.

assistant:
The first file does not contain data regarding failed banks by state; instead, it contains a set of questions, other ways to ask these questions, responses, and related GRI resource links. Let's move on to examine the next file. Please hold on while I check the content.

assistant:
The second file contains data related to COVID-19, with information on deaths, hospitalizations, and various testin

In [14]:
process_prompt("What was the last bank to fail?")

processing ...
user:
What was the last bank to fail?

assistant:
To determine the last bank to fail, I will sort the data by the "Closing Date" and find the most recent entry. I'll do this now for you.

assistant:
The last bank to fail, according to the data provided, was Citizens Bank, located in Sac City, Iowa. The bank failed on November 3, 2023.

If you have any more questions or need further assistance, please feel free to ask.



In [15]:
process_prompt("Which state has the most bank failures?")

processing ...
user:
Which state has the most bank failures?

assistant:
Based on the previously summarized data, I will identify the state with the most bank failures. Let me provide you with that information now.

assistant:
The state with the most bank failures, as per the data we have, is Georgia (GA), with a total of 93 failed banks.

Should you need any additional information or assistance, please let me know.



In [16]:
process_prompt("Which banks failed in Florida in between 2020-2023?")

processing ...
user:
Which banks failed in Florida in between 2020-2023?

assistant:
To find out which banks failed in Florida between 2020 and 2023, I will filter the data for banks located in Florida, and then further filter by the closing date to fall within the specified range from January 1, 2020, to December 31, 2023.

Let's do this now.

assistant:
Between 2020 and 2023, the following bank failed in Florida:

- First City Bank of Florida in Fort Walton Beach. The bank failed on October 16, 2020.

There are no other entries for failed banks in Florida within the specified date range in the dataset provided.

If you have any further questions or require more information, please let me know.



#### Cleanup

In [17]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for file in assistant_files:
    client.files.delete(file.id)

#### Using Functions

###### Get the latest stock price by ticker symbol using Yahoo Finance

In [20]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    return stock.history(period="1d")["Close"].iloc[-1]

#### Define the Assistant tools

In [ ]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Sends an email to a recipient(s).",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "The email(s) the email should be sent to."},
                    "content": {"type": "string", "description": "The content of the email."},
                },
                "required": ["to", "content"],
            },
        },
    },
]